In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, lag, when, last, sum, coalesce
from pyspark.sql.window import Window

# Initialize SparkSession
spark = SparkSession.builder.appName("SessionIdAllocation").getOrCreate()

# Sample data
data = [
    ("user1", "2024-09-13 10:00:00"),
    ("user1", "2024-09-13 10:06:00"),
    ("user1", "2024-09-13 10:40:00"),
    ("user2", "2024-09-13 10:05:00"),
    ("user2", "2024-09-13 10:09:00"),
    ("user2", "2024-09-13 10:13:00"),
    ("user2", "2024-09-13 10:17:00"),
    ("user2", "2024-09-13 10:21:00"),
    ("user2", "2024-09-13 10:25:00"),
    ("user2", "2024-09-13 10:29:00"),
    ("user2", "2024-09-13 10:33:00"),
    ("user2", "2024-09-13 10:35:00"),
    ("user2", "2024-09-13 10:39:00"),
    ("user2", "2024-09-13 10:43:00"),
    ("user2", "2024-09-13 10:47:00"),
    ("user2", "2024-09-13 10:59:00"),
]

# Create DataFrame
df = spark.createDataFrame(data, ["user_id", "timestamp"])
df = df.withColumn("timestamp", col("timestamp").cast("timestamp"))

# Define window specification for time difference calculation
windowSpec = Window.partitionBy("user_id").orderBy(col("timestamp"))

# Calculate time difference from previous event
df = df.withColumn("prev_timestamp", lag("timestamp").over(windowSpec))
df = df.withColumn("time_diff", unix_timestamp("timestamp")\
    - unix_timestamp("prev_timestamp"))

# Initialize a map to store the user ID and the last active session timestamp
user_last_active = {}

# Define a UDF to update the session ID based on the map
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, TimestampType

def update_session_id(user_id, timestamp, time_diff):
    global user_last_active
    if user_id not in user_last_active or time_diff is None or time_diff >= 300\
        or (timestamp - user_last_active[user_id]).seconds >= 1800:
        user_last_active[user_id] = timestamp
        return timestamp
    else:
        return user_last_active[user_id]

update_session_id_udf = udf(update_session_id, TimestampType())

# Apply the UDF to update the session ID
df = df.withColumn("session_id", update_session_id_udf(col("user_id"), col("timestamp"), col("time_diff")))

# Show the DataFrame with session IDs
# df.show(truncate=False)

# Define window specification for session duration calculation
windowSpec2 = Window.partitionBy("user_id", "session_id").orderBy(col("timestamp"))

# Calculate session duration
df = df.withColumn("session_duration", sum(coalesce(col("time_diff"), col("time_diff") * 0)).over(windowSpec2))

# Assign new session if duration exceeds 30 minutes
df = df.withColumn("final_session_id", when(col("session_duration") > 1800, col("timestamp"))
                                           .otherwise(col("session_id")))

# Fill null final session IDs with the last non-null final session ID within the window
df = df.withColumn("final_session_id", last("final_session_id", ignorenulls=True).over(windowSpec))

# Show the final DataFrame sorted by timestamp
df.select("user_id", "timestamp", "final_session_id").orderBy("user_id", "timestamp").show(truncate=False)

+-------+-------------------+-------------------+
|user_id|timestamp          |final_session_id   |
+-------+-------------------+-------------------+
|user1  |2024-09-13 10:00:00|2024-09-13 10:00:00|
|user1  |2024-09-13 10:06:00|2024-09-13 10:06:00|
|user1  |2024-09-13 10:40:00|2024-09-13 10:40:00|
|user2  |2024-09-13 10:05:00|2024-09-13 10:05:00|
|user2  |2024-09-13 10:09:00|2024-09-13 10:05:00|
|user2  |2024-09-13 10:13:00|2024-09-13 10:05:00|
|user2  |2024-09-13 10:17:00|2024-09-13 10:05:00|
|user2  |2024-09-13 10:21:00|2024-09-13 10:05:00|
|user2  |2024-09-13 10:25:00|2024-09-13 10:05:00|
|user2  |2024-09-13 10:29:00|2024-09-13 10:05:00|
|user2  |2024-09-13 10:33:00|2024-09-13 10:05:00|
|user2  |2024-09-13 10:35:00|2024-09-13 10:35:00|
|user2  |2024-09-13 10:39:00|2024-09-13 10:35:00|
|user2  |2024-09-13 10:43:00|2024-09-13 10:35:00|
|user2  |2024-09-13 10:47:00|2024-09-13 10:35:00|
|user2  |2024-09-13 10:59:00|2024-09-13 10:59:00|
+-------+-------------------+-------------------+
